In [40]:
# 0. Install dependencies (run once per environment)

%pip install fastai scikit-learn --quiet


Note: you may need to restart the kernel to use updated packages.


In [41]:
# 1. Imports and configuration

from pathlib import Path
import pandas as pd
import numpy as np
import torch
from torch import nn

from fastai.basics import *
import umap
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, ParameterSampler, cross_val_score
from sklearn.base import clone
from sklearn.calibration import CalibratedClassifierCV
from time import perf_counter

torch.manual_seed(42)
np.random.seed(42)

# Input datasets (normalized 0..1)
GENERATED_CSV_PATH = Path("Palettes/Generated/palette_export_generated.csv")
ADOBE_CSV_PATH = Path("Palettes/Adobe/adobe_palettes.csv")

# Rules-only encoder model (trained on generated data only)
MODEL_PATH = Path("trained_models/palette_autoencoder_rules_only.pkl")

# Permutation invariance
CLASS_TRAIN_PERM_AUGMENT = True
CLASS_TRAIN_PERM_K = 8   # additional random perms per palette during classifier training
CLASS_INFER_PERM_AVG = True
CLASS_INFER_PERM_K = 20  # additional random perms per palette during inference averaging
CLASS_PERM_SEED = 42

# Prototype confidence scaling
PROTO_CONF_SCALE = 10.0

# UMAP context
INCLUDE_GENERATED_IN_UMAP = True

# Outputs
OUT_UMAP_PRED_PATH = Path("out/adobe_umap_law_predictions.csv")
OUT_POPULARITY_PATH = Path("out/adobe_rule_popularity.csv")
OUT_METRICS_PATH = Path("out/generated_classifier_metrics.csv")

COLOR_COUNT = 5
SRGB_TO_LINEAR = True  # convert Adobe sRGB to linear before encoding
TEST_SIZE = 0.2

# Random Forest auto-tuning (ranges)
RF_AUTO_TUNE = True
RF_RANDOM_STATE = 42
RF_SEARCH_N_ITER = 24
RF_CV_FOLDS = 4
RF_SCORING = "accuracy"
RF_FIT_N_JOBS = 31  # use 31/32 threads, leave one for the system
RF_CV_N_JOBS = 1    # avoid nested parallelism

# Probability calibration for better confidence quality
RF_USE_PROB_CALIBRATION = True
RF_CALIBRATION_METHOD = "sigmoid"  # sigmoid or isotonic
RF_CALIBRATION_SPLIT = 0.15

RF_PARAM_GRID = {
    "n_estimators": [200, 400, 600, 800, 1000, 1200],
    "max_depth": [None, 12, 20, 30, 40],
    "min_samples_leaf": [1, 2, 3, 5],
    "max_features": ["sqrt", "log2", 0.5, 0.8],
    "class_weight": ["balanced", "balanced_subsample", None],
}


In [42]:
# 2. Define model class (required for load_learner)

class PaletteAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))


In [43]:
# 3. Utilities (data loading + encoding + permutations)

feature_cols = [f"{x}{i}" for i in range(1, COLOR_COUNT + 1) for x in ("r", "g", "b")]
expected_generated = ["law", "id_palette"] + [
    f"c{x}{i}" for i in range(1, COLOR_COUNT + 1) for x in ("r", "g", "b")
]
expected_adobe = [f"{x}{i}" for i in range(1, COLOR_COUNT + 1) for x in ("r", "g", "b")]


def _srgb_to_linear(arr):
    arr = np.clip(arr, 0.0, 1.0)
    return np.where(arr <= 0.04045, arr / 12.92, ((arr + 0.055) / 1.055) ** 2.4)


def _validate_normalized(df, cols, source_name):
    min_v = float(df[cols].min().min())
    max_v = float(df[cols].max().max())
    if min_v < 0.0 or max_v > 1.0:
        raise ValueError(
            f"{source_name} contains non-normalized values (min={min_v:.4f}, max={max_v:.4f}). "
            "Expected all color channels in [0, 1]."
        )


def load_generated_df(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Generated CSV not found: {path}")

    df = pd.read_csv(path)
    missing = [c for c in expected_generated if c not in df.columns]
    if missing:
        raise ValueError(f"Generated column mismatch for {path}. Missing {missing}, got {list(df.columns)}")

    df = df.drop(columns=[c for c in ("palette_name", "batch") if c in df.columns])
    rename_map = {f"c{x}{i}": f"{x}{i}" for i in range(1, COLOR_COUNT + 1) for x in ("r", "g", "b")}
    df = df.rename(columns=rename_map)

    _validate_normalized(df, feature_cols, f"Generated CSV ({path})")
    df[feature_cols] = df[feature_cols].astype("float32")
    df[["law", "id_palette"]] = df[["law", "id_palette"]].astype("int64")
    return df


def load_adobe_df(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Adobe CSV not found: {path}")

    df = pd.read_csv(path)
    if list(df.columns) != expected_adobe:
        raise ValueError(f"Adobe column mismatch for {path}. Expected {expected_adobe}, got {list(df.columns)}")

    df.insert(0, "id_palette", -1)
    df.insert(0, "law", -1)

    _validate_normalized(df, feature_cols, f"Adobe CSV ({path})")
    if SRGB_TO_LINEAR:
        df[feature_cols] = _srgb_to_linear(df[feature_cols].to_numpy(dtype="float32"))

    df[feature_cols] = df[feature_cols].astype("float32")
    df[["law", "id_palette"]] = df[["law", "id_palette"]].astype("int64")
    return df


def load_encoder(model_path: Path):
    if not model_path.exists():
        raise FileNotFoundError(f"Model not found at {model_path}. Run the training notebook first.")
    learner = load_learner(model_path)
    learner.model.eval()
    return learner.model


def encode_latents(model, data, batch_size=256):
    model.eval()
    device = next(model.parameters()).device
    zs = []
    with torch.no_grad():
        for start in range(0, len(data), batch_size):
            xb = torch.from_numpy(data[start:start + batch_size]).to(device)
            zs.append(model.encoder(xb).cpu())
    return torch.cat(zs).numpy()


def apply_palette_permutation(x_rgb15: np.ndarray, perm: np.ndarray) -> np.ndarray:
    x = x_rgb15.reshape(-1, COLOR_COUNT, 3)
    x = x[:, perm, :]
    return x.reshape(-1, COLOR_COUNT * 3)


def augment_training_permutations(x_rgb15: np.ndarray, y: np.ndarray, k: int, seed: int):
    rng = np.random.default_rng(seed)
    xs = [x_rgb15]
    ys = [y]
    for _ in range(k):
        perm = rng.permutation(COLOR_COUNT)
        xs.append(apply_palette_permutation(x_rgb15, perm))
        ys.append(y)
    return np.concatenate(xs, axis=0), np.concatenate(ys, axis=0)


def softmax_from_neg_dist(dist: np.ndarray, scale: float):
    scores = np.exp(-scale * dist)
    return scores / scores.sum(axis=1, keepdims=True)


In [44]:
# 4. Build train/eval sets (generated only) + encode with rules-only model

generated_path = GENERATED_CSV_PATH
adobe_path = ADOBE_CSV_PATH

print("Generated training source:", generated_path)
print("Adobe inference source:", adobe_path)
print("Encoder model:", MODEL_PATH)

gen_df = load_generated_df(generated_path)
adobe_df = load_adobe_df(adobe_path)

X_gen_base = gen_df[feature_cols].to_numpy(dtype="float32")
y_gen_base = gen_df["law"].to_numpy(dtype="int64")
X_adobe_base = adobe_df[feature_cols].to_numpy(dtype="float32")

if CLASS_TRAIN_PERM_AUGMENT:
    X_gen, y_gen = augment_training_permutations(
        X_gen_base,
        y_gen_base,
        k=CLASS_TRAIN_PERM_K,
        seed=CLASS_PERM_SEED,
    )
else:
    X_gen, y_gen = X_gen_base, y_gen_base

encoder_model = load_encoder(MODEL_PATH)
Z_gen = encode_latents(encoder_model, X_gen)
Z_gen_base = encode_latents(encoder_model, X_gen_base)

print("Generated base rows:", len(gen_df))
print("Generated train rows (after perm aug):", len(X_gen))
print("Adobe rows:", len(adobe_df))
print("Latent train shape:", Z_gen.shape)
print("Latent base shape:", Z_gen_base.shape)


Generated training source: Palettes\Generated\palette_export_generated.csv
Adobe inference source: Palettes\Adobe\adobe_palettes.csv
Encoder model: trained_models\palette_autoencoder_rules_only.pkl
Generated base rows: 7000
Generated train rows (after perm aug): 63000
Adobe rows: 1000
Latent train shape: (63000, 16)
Latent base shape: (7000, 16)


d:\OHM\Semester_3\fast_ai_study\.venv\Lib\site-packages\fastai\learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [45]:
# 5. Train classifier on generated labels only (auto-tuned RF + optional calibration)

X_train, X_valid, y_train, y_valid = train_test_split(
    Z_gen,
    y_gen,
    test_size=TEST_SIZE,
    random_state=42,
    stratify=y_gen,
)

# Split training into fit/calibration sets so validation remains unbiased
X_fit, X_cal, y_fit, y_cal = train_test_split(
    X_train,
    y_train,
    test_size=RF_CALIBRATION_SPLIT,
    random_state=RF_RANDOM_STATE,
    stratify=y_train,
)

rf_best_params = {}
rf_best_cv_score = float("-inf")

cv = StratifiedKFold(n_splits=RF_CV_FOLDS, shuffle=True, random_state=RF_RANDOM_STATE)
base_rf = RandomForestClassifier(
    random_state=RF_RANDOM_STATE,
    n_jobs=RF_FIT_N_JOBS,
)

if RF_AUTO_TUNE:
    sampled_params = list(ParameterSampler(RF_PARAM_GRID, n_iter=RF_SEARCH_N_ITER, random_state=RF_RANDOM_STATE))
    total = len(sampled_params)

    print(f"Starting RF search: {total} candidates, {RF_CV_FOLDS}-fold CV, scoring={RF_SCORING}")
    search_start = perf_counter()

    for i, params in enumerate(sampled_params, start=1):
        iter_start = perf_counter()

        model = clone(base_rf)
        model.set_params(**params)

        scores = cross_val_score(
            model,
            X_fit,
            y_fit,
            scoring=RF_SCORING,
            cv=cv,
            n_jobs=RF_CV_N_JOBS,
        )

        mean_score = float(np.mean(scores))
        std_score = float(np.std(scores))

        iter_elapsed = perf_counter() - iter_start
        elapsed = perf_counter() - search_start
        avg_per_iter = elapsed / i
        eta = avg_per_iter * (total - i)

        print(
            f"[RF Search] {i}/{total} | score={mean_score:.4f} +/- {std_score:.4f} | "
            f"iter={iter_elapsed:.1f}s | elapsed={elapsed/60:.1f}m | eta={eta/60:.1f}m"
        )

        if mean_score > rf_best_cv_score:
            rf_best_cv_score = mean_score
            rf_best_params = params
            print(f"  New best score: {rf_best_cv_score:.4f}")
            print(f"  Best params: {rf_best_params}")

    print("RF search finished.")
    print(f"Best CV score ({RF_SCORING}): {rf_best_cv_score:.4f}")
    print(f"Best RF params: {rf_best_params}")

    rf_model = RandomForestClassifier(
        **rf_best_params,
        random_state=RF_RANDOM_STATE,
        n_jobs=RF_FIT_N_JOBS,
    )
else:
    rf_best_params = {k: (v[0] if isinstance(v, list) else v) for k, v in RF_PARAM_GRID.items()}
    print("RF auto-tune disabled. Using first values from RF_PARAM_GRID:")
    print(rf_best_params)

    rf_model = RandomForestClassifier(
        **rf_best_params,
        random_state=RF_RANDOM_STATE,
        n_jobs=RF_FIT_N_JOBS,
    )

fit_start = perf_counter()
rf_model.fit(X_fit, y_fit)
fit_elapsed = perf_counter() - fit_start
print(f"Final RF fit done in {fit_elapsed:.1f}s")

# Optional probability calibration
if RF_USE_PROB_CALIBRATION:
    print(f"Calibrating probabilities (method={RF_CALIBRATION_METHOD}) on held-out calibration split...")
    clf = CalibratedClassifierCV(estimator=rf_model, method=RF_CALIBRATION_METHOD, cv='prefit')
    clf.fit(X_cal, y_cal)
    print("Calibration finished.")
else:
    clf = rf_model

# Evaluate on untouched validation split
y_valid_pred = clf.predict(X_valid)
valid_acc = float(accuracy_score(y_valid, y_valid_pred))

# Optional top-2 diagnostic
valid_proba = clf.predict_proba(X_valid)
valid_top2 = np.argsort(valid_proba, axis=1)[:, -2:]
valid_top2_labels = clf.classes_[valid_top2]
valid_top2_acc = float(np.mean([y_valid[i] in valid_top2_labels[i] for i in range(len(y_valid))]))

print(f"Validation accuracy: {valid_acc:.4f}")
print(f"Validation top-2 accuracy: {valid_top2_acc:.4f}")
print(classification_report(y_valid, y_valid_pred, digits=4))


Starting RF search: 24 candidates, 4-fold CV, scoring=accuracy
[RF Search] 1/24 | score=0.7819 +/- 0.0043 | iter=12.5s | elapsed=0.2m | eta=4.8m
  New best score: 0.7819
  Best params: {'n_estimators': 200, 'min_samples_leaf': 1, 'max_features': 0.8, 'max_depth': 12, 'class_weight': 'balanced'}
[RF Search] 2/24 | score=0.8049 +/- 0.0033 | iter=30.7s | elapsed=0.7m | eta=7.9m
  New best score: 0.8049
  Best params: {'n_estimators': 1200, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 12, 'class_weight': 'balanced_subsample'}
[RF Search] 3/24 | score=0.8427 +/- 0.0024 | iter=33.0s | elapsed=1.3m | eta=8.9m
  New best score: 0.8427
  Best params: {'n_estimators': 600, 'min_samples_leaf': 5, 'max_features': 0.5, 'max_depth': None, 'class_weight': 'balanced_subsample'}
[RF Search] 4/24 | score=0.8545 +/- 0.0024 | iter=55.3s | elapsed=2.2m | eta=11.0m
  New best score: 0.8545
  Best params: {'n_estimators': 1200, 'min_samples_leaf': 3, 'max_features': 0.5, 'max_depth': None, 'cl

d:\OHM\Semester_3\fast_ai_study\.venv\Lib\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


Calibration finished.
Validation accuracy: 0.8873
Validation top-2 accuracy: 0.9610
              precision    recall  f1-score   support

           0     0.8643    0.9517    0.9059      1800
           1     0.8755    0.9217    0.8980      1800
           2     0.9747    0.8972    0.9343      1800
           3     0.8284    0.7967    0.8122      1800
           4     0.8329    0.8111    0.8218      1800
           5     0.8936    0.9094    0.9014      1800
           6     0.9497    0.9233    0.9363      1800

    accuracy                         0.8873     12600
   macro avg     0.8884    0.8873    0.8871     12600
weighted avg     0.8884    0.8873    0.8871     12600



In [46]:
# 6. Predict Adobe rules + UMAP + combined output CSV

law_ids = np.array(sorted(np.unique(y_gen)))
centroids = np.stack([Z_gen[y_gen == law].mean(axis=0) for law in law_ids], axis=0)
c_norm = centroids / np.linalg.norm(centroids, axis=1, keepdims=True)

# Build permutation list for inference averaging
if CLASS_INFER_PERM_AVG:
    rng = np.random.default_rng(CLASS_PERM_SEED)
    infer_perms = [np.arange(COLOR_COUNT)] + [rng.permutation(COLOR_COUNT) for _ in range(CLASS_INFER_PERM_K)]
else:
    infer_perms = [np.arange(COLOR_COUNT)]

class_proba_sum = None
proto_proba_sum = None
Z_adobe_sum = None

for perm in infer_perms:
    Xp = apply_palette_permutation(X_adobe_base, perm)
    Zp = encode_latents(encoder_model, Xp)

    # For UMAP and optional downstream use
    Z_adobe_sum = Zp if Z_adobe_sum is None else (Z_adobe_sum + Zp)

    # -------- Classifier-based probabilities --------
    class_proba = clf.predict_proba(Zp)
    class_proba_sum = class_proba if class_proba_sum is None else (class_proba_sum + class_proba)

    # -------- Prototype-based probabilities --------
    z_norm = Zp / np.linalg.norm(Zp, axis=1, keepdims=True)
    cos_sim = z_norm @ c_norm.T
    cos_dist = 1.0 - cos_sim
    proto_proba = softmax_from_neg_dist(cos_dist, scale=PROTO_CONF_SCALE)
    proto_proba_sum = proto_proba if proto_proba_sum is None else (proto_proba_sum + proto_proba)

num_passes = len(infer_perms)
class_proba_avg = class_proba_sum / num_passes
proto_proba_avg = proto_proba_sum / num_passes
Z_adobe = Z_adobe_sum / num_passes

# -------- Classifier-based predictions (prefixed with class_) --------
class_ids = clf.classes_
class_top2_idx = np.argsort(class_proba_avg, axis=1)[:, -2:]
class_second_idx = class_top2_idx[:, 0]
class_first_idx = class_top2_idx[:, 1]

class_pred_law = class_ids[class_first_idx]
class_pred_confidence = class_proba_avg[np.arange(len(class_proba_avg)), class_first_idx]
class_second_law = class_ids[class_second_idx]
class_second_confidence = class_proba_avg[np.arange(len(class_proba_avg)), class_second_idx]

# -------- Prototype predictions (clean names, no prefix) --------
proto_top2_idx = np.argsort(proto_proba_avg, axis=1)[:, -2:]
proto_second_idx = proto_top2_idx[:, 0]
proto_first_idx = proto_top2_idx[:, 1]

pred_law = law_ids[proto_first_idx]
pred_confidence = proto_proba_avg[np.arange(len(proto_proba_avg)), proto_first_idx]
second_law = law_ids[proto_second_idx]
second_confidence = proto_proba_avg[np.arange(len(proto_proba_avg)), proto_second_idx]

# Build a combined latent set for UMAP context
if INCLUDE_GENERATED_IN_UMAP:
    Z_all = np.vstack([Z_gen_base, Z_adobe])
else:
    Z_all = Z_adobe

umap_3d = umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.05, metric="euclidean", random_state=42)
Z_all_3d = umap_3d.fit_transform(Z_all)

if INCLUDE_GENERATED_IN_UMAP:
    n_gen = len(Z_gen_base)
    gen_umap = Z_all_3d[:n_gen]
    adobe_umap = Z_all_3d[n_gen:]

    # Generated rows (for context), keep known law labels
    gen_out = pd.DataFrame({
        "umap_x": gen_umap[:, 0],
        "umap_y": gen_umap[:, 1],
        "umap_z": gen_umap[:, 2],
        "law": y_gen_base,
        "pred_law": y_gen_base,
        "pred_confidence": np.ones(n_gen, dtype="float32"),
        "second_law": np.full(n_gen, -1, dtype="int64"),
        "second_confidence": np.zeros(n_gen, dtype="float32"),
        "class_pred_law": y_gen_base,
        "class_pred_confidence": np.ones(n_gen, dtype="float32"),
        "class_second_law": np.full(n_gen, -1, dtype="int64"),
        "class_second_confidence": np.zeros(n_gen, dtype="float32"),
        "adobe": np.zeros(n_gen, dtype="int64"),
    })

    # Adobe rows with predictions
    adobe_out = pd.DataFrame({
        "umap_x": adobe_umap[:, 0],
        "umap_y": adobe_umap[:, 1],
        "umap_z": adobe_umap[:, 2],
        "law": np.full(len(adobe_df), -1, dtype="int64"),
        "pred_law": pred_law,
        "pred_confidence": pred_confidence,
        "second_law": second_law,
        "second_confidence": second_confidence,
        "class_pred_law": class_pred_law,
        "class_pred_confidence": class_pred_confidence,
        "class_second_law": class_second_law,
        "class_second_confidence": class_second_confidence,
        "adobe": np.ones(len(adobe_df), dtype="int64"),
    })

    out_df = pd.concat([gen_out, adobe_out], ignore_index=True)
else:
    # Adobe-only output mode
    out_df = pd.DataFrame({
        "umap_x": Z_all_3d[:, 0],
        "umap_y": Z_all_3d[:, 1],
        "umap_z": Z_all_3d[:, 2],
        "law": np.full(len(adobe_df), -1, dtype="int64"),
        "pred_law": pred_law,
        "pred_confidence": pred_confidence,
        "second_law": second_law,
        "second_confidence": second_confidence,
        "class_pred_law": class_pred_law,
        "class_pred_confidence": class_pred_confidence,
        "class_second_law": class_second_law,
        "class_second_confidence": class_second_confidence,
        "adobe": np.ones(len(adobe_df), dtype="int64"),
    })

# Popularity summary from Adobe-only clean (prototype) predictions
popularity_df = (
    pd.Series(pred_law)
    .value_counts(dropna=False)
    .rename_axis("law")
    .reset_index(name="count")
)
popularity_df["share"] = popularity_df["count"] / popularity_df["count"].sum()

OUT_UMAP_PRED_PATH.parent.mkdir(parents=True, exist_ok=True)
out_df.to_csv(OUT_UMAP_PRED_PATH, index=False)
popularity_df.to_csv(OUT_POPULARITY_PATH, index=False)

metrics_df = pd.DataFrame([
    {
        "validation_accuracy": valid_acc,
        "validation_top2_accuracy": valid_top2_acc,
        "classifier_type": "RandomForestClassifier",
        "rf_auto_tune": RF_AUTO_TUNE,
        "rf_scoring": RF_SCORING,
        "rf_search_n_iter": RF_SEARCH_N_ITER,
        "rf_cv_folds": RF_CV_FOLDS,
        "rf_best_cv_score": rf_best_cv_score,
        "rf_best_params": str(rf_best_params),
        "rf_param_grid": str(RF_PARAM_GRID),
        "rf_fit_n_jobs": RF_FIT_N_JOBS,
        "rf_cv_n_jobs": RF_CV_N_JOBS,
        "rf_use_prob_calibration": RF_USE_PROB_CALIBRATION,
        "rf_calibration_method": RF_CALIBRATION_METHOD,
        "rf_calibration_split": RF_CALIBRATION_SPLIT,
        "test_size": TEST_SIZE,
        "generated_path": str(generated_path),
        "model_path": str(MODEL_PATH),
        "adobe_path": str(adobe_path),
        "train_perm_augment": CLASS_TRAIN_PERM_AUGMENT,
        "train_perm_k": CLASS_TRAIN_PERM_K,
        "infer_perm_avg": CLASS_INFER_PERM_AVG,
        "infer_perm_k": CLASS_INFER_PERM_K,
        "perm_seed": CLASS_PERM_SEED,
        "perm_passes_used": num_passes,
        "include_generated_in_umap": INCLUDE_GENERATED_IN_UMAP,
        "generated_rows_in_output": int(len(gen_df) if INCLUDE_GENERATED_IN_UMAP else 0),
        "adobe_rows_in_output": int(len(adobe_df)),
    }
])
metrics_df.to_csv(OUT_METRICS_PATH, index=False)

print("Saved combined UMAP predictions:", OUT_UMAP_PRED_PATH)
print("Saved popularity:", OUT_POPULARITY_PATH)
print("Saved metrics:", OUT_METRICS_PATH)
print(f"Inference permutation passes: {num_passes}")
out_df.head()


d:\OHM\Semester_3\fast_ai_study\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved combined UMAP predictions: out\adobe_umap_law_predictions.csv
Saved popularity: out\adobe_rule_popularity.csv
Saved metrics: out\generated_classifier_metrics.csv
Inference permutation passes: 21


,umap_x,umap_y,umap_z,law,pred_law,pred_confidence,second_law,second_confidence,class_pred_law,class_pred_confidence,class_second_law,class_second_confidence,adobe
0,4.464985,10.679130,2.334833,0,0,1.0,-1,0.0,0,1.0,-1,0.0,0
1,3.249228,10.310564,2.228700,0,0,1.0,-1,0.0,0,1.0,-1,0.0,0
2,2.305106,11.541363,6.234135,0,0,1.0,-1,0.0,0,1.0,-1,0.0,0
3,3.172910,9.486996,2.014287,0,0,1.0,-1,0.0,0,1.0,-1,0.0,0
4,4.005281,9.262683,2.211741,0,0,1.0,-1,0.0,0,1.0,-1,0.0,0
